In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama3-70b-8192",
    temperature=0,
    groq_api_key='gsk_GO9Z7'

)


response = llm.invoke("The first person to walk on moon was...")
print(response.content)


That's an easy one! The first person to walk on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he took his first steps on the moon.


In [3]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-37695")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Bewirb dich für Manager Software Engineering, ITC

JobsucheNavigation überspringenJobsucheNIKE, INC. JOBSBefristete JobsJoin The Talent CommunityUnser Leben bei NikeÜbersichtLeistungenMarkenÜbersichtJordanConverseTeamsÜbersichtAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public Affairs
Human ResourcesData & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyStandorteÜbersichtNike WHQNike New York HQEHQ: Hilversum, NiederlandeELC: Laakdal, BelgiumGreater China HQDiversität, Gleichstellung und InklusionÜbersichtMilitärBehinderung und InklusionInklusion der indigenen BevölkerungPraktikaTechnologyManager Software Engineering, ITCKarnataka, Ind

In [4]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION
    The scraped test is from the career's page website.
    Your job is to extract the job postings and return them in JSON format containing the following keys: `role`, `experience`, `skills` and `description`.
    Only return the valid JSON object. NO PREAMBLE.
    ### VALID JSON (NO PREAMBLE):
    """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

{
"role": "Manager Software Engineering, ITC",
"experience": "A minimum of 5 years management experience (as direct people manager or equivalent of managing engineers) of high performing software engineering teams and total 12+ years of experience in software development.",
"skills": ["Academic background in computer science (BS or MS) or equivalent work experience", "Leadership in a highly collaborative, multi-discipline development team environment", "Exposure to backend development skills in Java/Spring", "Experience of building microservices using cloud services to build an integrated application (AWS preferred)", "Exposure to tuning relational database (MySQL) and non-relation DB (MongoDB, Dynamo DB)", "Experience with building data-intensive web applications using modern UI frameworks like React or Polymer", "Experience working with Product Managers, other development teams, and technology platform teams to accelerate software development and increase quality", "Experience in goa

In [6]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Manager Software Engineering, ITC',
 'experience': 'A minimum of 5 years management experience (as direct people manager or equivalent of managing engineers) of high performing software engineering teams and total 12+ years of experience in software development.',
 'skills': ['Academic background in computer science (BS or MS) or equivalent work experience',
  'Leadership in a highly collaborative, multi-discipline development team environment',
  'Exposure to backend development skills in Java/Spring',
  'Experience of building microservices using cloud services to build an integrated application (AWS preferred)',
  'Exposure to tuning relational database (MySQL) and non-relation DB (MongoDB, Dynamo DB)',
  'Experience with building data-intensive web applications using modern UI frameworks like React or Polymer',
  'Experience working with Product Managers, other development teams, and technology platform teams to accelerate software development and increase quality',
  'Ex

In [ ]:
import pandas as pd 

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [7]:

import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _,row in df.iterrows():
        collection.add(documents = row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [10]:
job = json_res
links = collection.query(query_texts=job['skills'],n_results=2).get('metadatas')
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/kotlin-backend-portfolio'},
  {'links': 'https://example.com/java-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/xamarin-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],


In [11]:
job['skills']

['Academic background in computer science (BS or MS) or equivalent work experience',
 'Leadership in a highly collaborative, multi-discipline development team environment',
 'Exposure to backend development skills in Java/Spring',
 'Experience of building microservices using cloud services to build an integrated application (AWS preferred)',
 'Exposure to tuning relational database (MySQL) and non-relation DB (MongoDB, Dynamo DB)',
 'Experience with building data-intensive web applications using modern UI frameworks like React or Polymer',
 'Experience working with Product Managers, other development teams, and technology platform teams to accelerate software development and increase quality',
 'Experience in goal settings, mentoring hiring and ramping up the teams',
 'Comfortable working within an exciting fast-paced environment by reducing ambiguity and building trust with the team',
 'Willingness to explore new technologies and creative solutions',
 'Strong at estimating work, plann

In [12]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Software Engineering Leadership for Your Digital Asset Management System

Dear Hiring Manager,

I came across the job description for a Manager Software Engineering, ITC at Nike, and I'm excited to introduce AtliQ, an AI & Software Consulting company that can help you fulfill your software engineering needs. Our team of experts has a proven track record of delivering scalable, high-performing software solutions that meet the demands of fast-paced environments.

With our expertise in software development, we can provide engineering leadership and best practices to continuously build new platforms and deliver working software. Our team is well-versed in backend development skills in Java/Spring, building microservices using cloud services, and tuning relational and non-relational databases (MySQL, MongoDB, Dynamo DB). We also have experience with modern UI frameworks like React or Polymer and building data-intensive web applications.

Our DevOps capabilities ensure seamle